In [1]:
import torch

In [2]:
%%latex
$$C_{ij}=C(o_{ij})=-\bar{P_{ij}}log(P_{ij})-(1-\bar{P_{ij}})log(1-P_{ij})$$

<IPython.core.display.Latex object>

In [3]:
%%latex
$$o_{ij}=f(x_i)-f(x_j)$$

<IPython.core.display.Latex object>

In [4]:
%%latex
$$P_{ij}=\frac{e^{o_{ij}}}{1+e^{o_{ij}}}$$

<IPython.core.display.Latex object>

In [5]:
# torch.sigmoid()

In [6]:
%%latex
$$\text{out}_{i} = \frac{1}{1 + e^{-\text{input}_{i}}}$$

<IPython.core.display.Latex object>

In [7]:
class RankNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1)
        )
        
        self.out_activation = torch.nn.Sigmoid()
        
    def forward(self, input_1, input_2):
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)
        
        logits_diff = logits_1 - logits_2
        out = self.out_activation(logits_diff)
        
        return out
    
    def predict(self, inp):
        logits = self.model(inp)
        return logits

In [8]:
ranknet_model = RankNet(num_input_features=10)

In [9]:
inp_1, inp_2 = torch.rand(4, 10), torch.rand(4, 10)
# batch_size x input_dim
inp_1

tensor([[0.8490, 0.8237, 0.1802, 0.9960, 0.2599, 0.2909, 0.5181, 0.0925, 0.7493,
         0.3626],
        [0.4191, 0.4184, 0.6492, 0.3494, 0.8826, 0.5463, 0.2992, 0.7878, 0.3409,
         0.6044],
        [0.2255, 0.7258, 0.3763, 0.1904, 0.5593, 0.0947, 0.5973, 0.9100, 0.4831,
         0.8721],
        [0.4308, 0.4759, 0.3197, 0.1187, 0.4245, 0.7637, 0.7419, 0.7369, 0.6362,
         0.5436]])

In [10]:
preds = ranknet_model(inp_1, inp_2)
preds

tensor([[0.5096],
        [0.5000],
        [0.5089],
        [0.4744]], grad_fn=<SigmoidBackward0>)

In [11]:
first_linear_layer = ranknet_model.model[0]

In [12]:
first_linear_layer.weight.grad

In [13]:
criterion = torch.nn.BCELoss()
loss = criterion(preds, torch.ones_like(preds))
loss.backward()

In [14]:
first_linear_layer.weight.grad

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.2917e-02,  1.9002e-02,  4.7833e-03,  2.5640e-03, -1.0699e-02,
          6.0162e-04, -1.2418e-02,  1.1490e-02,  3.5278e-05,  1.0444e-03],
        [ 8.0022e-03,  8.2669e-03,  7.1168e-03,  4.4491e-04,  5.7428e-03,
          1.3951e-03,  3.5157e-03,  1.0438e-02,  8.6375e-03,  7.0413e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.7366e-02,  2.5546e-02,  6.4307e-03,  3.4470e-03, -1.4384e-02,
          8.0882e-04, -1.6695e-02,  1.5448e-02,  4.7430e-05,  1.4041e-03],
        [ 2.1856e-02,  3.2150e-02,  8.0933e-03,  4.3382e-03, -1.8103e-02,
          1.0179e-03, -2.1012e-02,  1.9441e-02,  5.9694e-05,  1.7671e-03],
        [-2.7975e-03, -8.0262e-03,  4.1042e-03, -3.4241e-03,  1.1900e-02,
         -1.2115e-02,  2.9405e-0

In [15]:
ranknet_model.zero_grad()

# ListNet

In [16]:
from itertools import combinations
import numpy as np

from utils import ndcg, num_swapped_pairs

In [17]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )
        
    def forward(self, input_1):
        logits = self.model(input_1)
        return logits

In [18]:
%%latex
$$CE = -\sum ^{N}_{j=1} (P_y^i(j) * log(P_z^i(j)))$$

<IPython.core.display.Latex object>

In [19]:
%%latex
$$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$$

<IPython.core.display.Latex object>

In [20]:
def listnet_ce_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i))

def listnet_kl_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i/P_y_i))

def make_dataset(N_train, N_valid, vector_dim):
    fake_weights = torch.randn(vector_dim, 1)
    
    X_train = torch.randn(N_train, vector_dim)
    X_valid = torch.randn(N_valid, vector_dim)
    
    ys_train_score = torch.mm(X_train, fake_weights)
    ys_train_score += torch.randn_like(ys_train_score)
    
    ys_valid_score = torch.mm(X_valid, fake_weights)
    ys_valid_score += torch.randn_like(ys_valid_score)
    
#     bins = [-1, 1] # 3 relevances
    bins = [-1, 0, 1, 2] # 5 relevances
    
    ys_train_rel = torch.Tensor(
        np.digitize(ys_train_score.clone().detach().numpy(), bins=bins)
    )
    ys_valid_rel = torch.Tensor(
        np.digitize(ys_valid_score.clone().detach().numpy(), bins=bins)
    )
    
    return X_train, X_valid, ys_train_rel, ys_valid_rel

In [21]:
N_train = 1000
N_valid = 500

vector_dim = 100
epochs = 2

batch_size = 16

X_train, X_valid, ys_train, ys_valid = make_dataset(N_train, N_valid, vector_dim)

net = ListNet(num_input_features=vector_dim)
opt = torch.optim.Adam(net.parameters())

In [22]:
torch.unique(ys_train)

tensor([0., 1., 2., 3., 4.])

In [23]:
for epoch in range(epochs):
    idx = torch.randperm(N_train)
    
    X_train = X_train[idx]
    ys_train = ys_train[idx]
    
    cur_batch = 0
    for it in range(N_train // batch_size):
        batch_X = X_train[cur_batch: cur_batch + batch_size]
        batch_ys = ys_train[cur_batch: cur_batch + batch_size]
        cur_batch += batch_size
        
        
        opt.zero_grad()
        if len(batch_X) > 0:
            batch_pred = net(batch_X)
            # batch_loss = listnet_kl_loss(batch_ys, batch_pred)
            batch_loss = listnet_ce_loss(batch_ys, batch_pred)
            batch_loss.backward(retain_graph=True)
            opt.step()
            
        if it % 10 == 0:
            with torch.no_grad():
                valid_pred = net(X_valid)
                valid_swapped_pairs = num_swapped_pairs(ys_valid, valid_pred)
                ndcg_score = ndcg(ys_valid, valid_pred)
            print(f"epoch: {epoch + 1}.\tNumber of swapped pairs: "
                  f"{valid_swapped_pairs} / {N_valid * (N_valid - 1) // 2}\t"
                  f"nDCG: {ndcg_score:.4f}"
                 )
            
    

epoch: 1.	Number of swapped pairs: 44944 / 124750	nDCG: 0.8155
epoch: 1.	Number of swapped pairs: 42707 / 124750	nDCG: 0.8360
epoch: 1.	Number of swapped pairs: 40438 / 124750	nDCG: 0.8449
epoch: 1.	Number of swapped pairs: 37929 / 124750	nDCG: 0.8583
epoch: 1.	Number of swapped pairs: 35563 / 124750	nDCG: 0.8743
epoch: 1.	Number of swapped pairs: 32976 / 124750	nDCG: 0.8847
epoch: 1.	Number of swapped pairs: 30366 / 124750	nDCG: 0.9001
epoch: 2.	Number of swapped pairs: 29870 / 124750	nDCG: 0.9021
epoch: 2.	Number of swapped pairs: 27481 / 124750	nDCG: 0.9119
epoch: 2.	Number of swapped pairs: 25316 / 124750	nDCG: 0.9217
epoch: 2.	Number of swapped pairs: 23164 / 124750	nDCG: 0.9314
epoch: 2.	Number of swapped pairs: 21406 / 124750	nDCG: 0.9408
epoch: 2.	Number of swapped pairs: 19872 / 124750	nDCG: 0.9467
epoch: 2.	Number of swapped pairs: 18418 / 124750	nDCG: 0.9530
